In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import torch
from torchvision import models

from datasets import ImagenetDataset
from fooling_model import VGG16FoolingModel, VGG19FoolingModel, ResNet50FoolingModel
from utils import construct_perturbation, calculate_fooling_rate, process_image

In [2]:
np.random.seed(2)
torch.manual_seed(2)
torch.backends.cudnn.deterministic = True

In [3]:
BATCH_SIZE = 64
MAX_PERTURB_NORM = 10
p = np.inf
q = 10

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

imgs_dir = 'imgs'
imgs_paths = [os.path.join(imgs_dir, img_name) for img_name in os.listdir(imgs_dir)]
train_paths = np.random.choice(imgs_paths, size=BATCH_SIZE, replace=False)
val_paths = list(set(imgs_paths) - set(train_paths))
train_dataset = ImagenetDataset(train_paths, device)
val_dataset = ImagenetDataset(val_paths, device)

In [7]:
fooling_models = {
    'vgg16': VGG16FoolingModel(models.vgg16(pretrained=True)).to(device),
    'vgg19': VGG19FoolingModel(models.vgg19(pretrained=True)).to(device),
    'resnet50': ResNet50FoolingModel(models.resnet50(pretrained=True)).to(device)
}
for model in fooling_models.values():
    model.eval()

fooling_layers = {
    'vgg16': fooling_models['vgg16'].LAYERS_IDS['block2_pool'],
    'vgg19': fooling_models['vgg19'].LAYERS_IDS['block2_pool'],
    'resnet50': fooling_models['resnet50'].LAYERS_IDS['conv1'],
}

In [8]:
perturbations = {}
train_batch = train_dataset[:]

for model_name, model in fooling_models.items():
    perturbations[model_name] = construct_perturbation(
        model=model,
        layer_id=fooling_layers[model_name],
        train_batch=train_batch,
        perturb_norm=MAX_PERTURB_NORM,
        p=p,
        q=q
    )

In [ ]:
fooling_rates = {}
for model_name, model in fooling_models.items():
    fooling_rates[model_name] = {
        perturb_model_name: calculate_fooling_rate(model, perturbations[perturb_model_name], val_dataset)
        for perturb_model_name in perturbations.keys()
    }

In [16]:
df = pd.DataFrame(fooling_rates)
df.head()

,resnet50,vgg16,vgg19
resnet50,0.503425,0.629527,0.567388
vgg16,0.371454,0.577945,0.564283
vgg19,0.368710,0.584776,0.563942
